In [2]:
import tensorflow as tf

# create a node called Operation（Op）
a = tf.constant(5, name="input_a")

# create a node called Operation（Op）
b = tf.constant(3, name="input_b")

# tensorflow 创建节点时已经包含了相应的 Op 用于提供计算需要的输入
# tensorflow 会自动绘制必要的连接
c = tf.multiply(a, b, name="mul_c")
d = tf.add(a, b, name="add_d")

# 这里定义了数据流的终点 e
e = tf.add(c, d, name="add_e")

### 上面定义了若干节点，完成了一个小规模数据流图的完整定义

### Session 对象在运行时负责对数据流图进行监督，并且是运行数据流图的主要接口。

当然，tf.Session 有一个与之十分相近的变体 tf.InteractiveSession。


In [3]:
sess = tf.Session()
sess.run(e)

# 习惯上会用 tf.summary.FileWriter 保存来自数据流图的数据和概括统计量。
writer = tf.summary.FileWriter("./my_graph", sess.graph)

writer.close()
sess.close()

In [28]:
f = tf.constant([3, 5], name="input_f")
print(f)

import numpy as np

# 视为 0 阶张量或”标量“
t_0 = 50
print(np.array(t_0).shape)
# 视为 1 阶张量或“向量”
t_1 = ["apple", "peach", "grape"]
print(np.array(t_1).shape)
# 视为 2 阶张量或“矩阵”
t_2 = [
    [True, False, False],
    [True, False, False],
    [True, False, False],
]
print(np.array(t_2).shape)
# 视为 3 阶张量
t_3 = [[[0, 0], [0, 1], [0, 2]],
      [[1, 0], [1, 1], [1, 2]],
      [[2, 0], [2, 1], [2, 2]]]

print(np.array(t_3).shape)
print(np.array(t_3).flags)

Tensor("input_f_11:0", shape=(2,), dtype=int32)
()
(3,)
(3, 3)
(3, 3, 2)
  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False
